This first cell does setup. It takes data by reading `uni_data.csv` into a Pandas dataframe, but if you combine this with `scrap_wiki.ipynb` you could probably just read it directly.

In [63]:
# Run `pip install ultimate_sitemap_parser` first
from usp.tree import sitemap_tree_for_homepage
from pprint import pprint
import pandas as pd

In [64]:
def getSitemap(dfRow):
    return sitemap_tree_for_homepage(dfRow['url'])

In [68]:
# Load university dataframe, first 50 rows only to keep this manageable
data = pd.read_csv('uni_data.csv')
data = data.head(50)

In [69]:
# Apply getSitemap() to every row of data
# getSitemap() returns a sitemap tree object, which is stored in a new column
data['sitemap tree'] = data.apply(getSitemap, axis=1)

2020-06-27 14:57:43,175 INFO usp.fetch_parse [15023/MainThread]: Fetching level 0 sitemap from https://web.mit.edu/robots.txt...
2020-06-27 14:57:43,176 INFO usp.helpers [15023/MainThread]: Fetching URL https://web.mit.edu/robots.txt...
2020-06-27 14:57:44,096 INFO usp.fetch_parse [15023/MainThread]: Parsing sitemap from URL https://web.mit.edu/robots.txt...
2020-06-27 14:57:44,098 INFO usp.fetch_parse [15023/MainThread]: Fetching level 0 sitemap from https://web.mit.edu/sitemap_index.xml.gz...
2020-06-27 14:57:44,099 INFO usp.helpers [15023/MainThread]: Fetching URL https://web.mit.edu/sitemap_index.xml.gz...
2020-06-27 14:57:44,324 WARNING usp.helpers [15023/MainThread]: Request for URL https://web.mit.edu/sitemap_index.xml.gz failed: 404 Not Found
2020-06-27 14:57:44,325 INFO usp.helpers [15023/MainThread]: Not retrying for URL https://web.mit.edu/sitemap_index.xml.gz
2020-06-27 14:57:44,326 INFO usp.fetch_parse [15023/MainThread]: Fetching level 0 sitemap from https://web.mit.edu/s

2020-06-27 14:57:49,340 INFO usp.helpers [15023/MainThread]: Not retrying for URL http://harvard.edu/sitemap-index.xml.gz
2020-06-27 14:57:49,342 INFO usp.fetch_parse [15023/MainThread]: Fetching level 0 sitemap from http://harvard.edu/sitemap.xml.gz...
2020-06-27 14:57:49,343 INFO usp.helpers [15023/MainThread]: Fetching URL http://harvard.edu/sitemap.xml.gz...
2020-06-27 14:57:49,675 WARNING usp.helpers [15023/MainThread]: Request for URL http://harvard.edu/sitemap.xml.gz failed: 404 Not Found
2020-06-27 14:57:49,677 INFO usp.helpers [15023/MainThread]: Not retrying for URL http://harvard.edu/sitemap.xml.gz
2020-06-27 14:57:49,677 INFO usp.fetch_parse [15023/MainThread]: Fetching level 0 sitemap from http://harvard.edu/admin/config/search/xmlsitemap...
2020-06-27 14:57:49,678 INFO usp.helpers [15023/MainThread]: Fetching URL http://harvard.edu/admin/config/search/xmlsitemap...
2020-06-27 14:57:49,919 WARNING usp.helpers [15023/MainThread]: Request for URL http://harvard.edu/admin/con

2020-06-27 14:57:54,918 ERROR usp.fetch_parse [15023/MainThread]: Parsing sitemap from URL http://stanford.edu/sitemap_index.xml failed: Unsupported root element 'html'.
2020-06-27 14:57:54,918 INFO usp.fetch_parse [15023/MainThread]: Fetching level 0 sitemap from http://stanford.edu/.sitemap.xml...
2020-06-27 14:57:54,919 INFO usp.helpers [15023/MainThread]: Fetching URL http://stanford.edu/.sitemap.xml...
2020-06-27 14:57:55,093 WARNING usp.helpers [15023/MainThread]: Request for URL http://stanford.edu/.sitemap.xml failed: 404 Not Found
2020-06-27 14:57:55,093 INFO usp.helpers [15023/MainThread]: Not retrying for URL http://stanford.edu/.sitemap.xml
2020-06-27 14:57:55,094 INFO usp.fetch_parse [15023/MainThread]: Fetching level 0 sitemap from http://stanford.edu/sitemap...
2020-06-27 14:57:55,095 INFO usp.helpers [15023/MainThread]: Fetching URL http://stanford.edu/sitemap...
2020-06-27 14:57:55,573 INFO usp.fetch_parse [15023/MainThread]: Parsing sitemap from URL http://stanford.ed

2020-06-27 14:58:13,310 INFO usp.helpers [15023/MainThread]: Not retrying for URL http://www.purdue.edu/sitemap_index.xml.gz
2020-06-27 14:58:13,311 INFO usp.fetch_parse [15023/MainThread]: Fetching level 0 sitemap from http://www.purdue.edu/sitemap-index.xml...
2020-06-27 14:58:13,312 INFO usp.helpers [15023/MainThread]: Fetching URL http://www.purdue.edu/sitemap-index.xml...
2020-06-27 14:58:13,612 WARNING usp.helpers [15023/MainThread]: Request for URL http://www.purdue.edu/sitemap-index.xml failed: 404 Not Found
2020-06-27 14:58:13,613 INFO usp.helpers [15023/MainThread]: Not retrying for URL http://www.purdue.edu/sitemap-index.xml
2020-06-27 14:58:13,614 INFO usp.fetch_parse [15023/MainThread]: Fetching level 0 sitemap from http://www.purdue.edu/sitemap/sitemap-index.xml...
2020-06-27 14:58:13,615 INFO usp.helpers [15023/MainThread]: Fetching URL http://www.purdue.edu/sitemap/sitemap-index.xml...
2020-06-27 14:58:13,907 WARNING usp.helpers [15023/MainThread]: Request for URL http:

2020-06-27 14:58:21,944 INFO usp.fetch_parse [15023/MainThread]: Fetching level 0 sitemap from http://cornell.edu/sitemap_index.xml...
2020-06-27 14:58:21,944 INFO usp.helpers [15023/MainThread]: Fetching URL http://cornell.edu/sitemap_index.xml...
2020-06-27 14:58:22,467 WARNING usp.helpers [15023/MainThread]: Request for URL http://cornell.edu/sitemap_index.xml failed: 404 Not Found
2020-06-27 14:58:22,469 INFO usp.helpers [15023/MainThread]: Not retrying for URL http://cornell.edu/sitemap_index.xml
2020-06-27 14:58:22,470 INFO usp.fetch_parse [15023/MainThread]: Fetching level 0 sitemap from http://cornell.edu/.sitemap.xml...
2020-06-27 14:58:22,471 INFO usp.helpers [15023/MainThread]: Fetching URL http://cornell.edu/.sitemap.xml...
2020-06-27 14:58:22,841 WARNING usp.helpers [15023/MainThread]: Request for URL http://cornell.edu/.sitemap.xml failed: 403 Forbidden
2020-06-27 14:58:22,842 INFO usp.helpers [15023/MainThread]: Not retrying for URL http://cornell.edu/.sitemap.xml
2020-0

...

2020-06-27 15:06:26,524 INFO usp.fetch_parse [15023/MainThread]: Fetching level 0 sitemap from http://www.cmu.edu/sitemap_index.xml...
2020-06-27 15:06:26,525 INFO usp.helpers [15023/MainThread]: Fetching URL http://www.cmu.edu/sitemap_index.xml...
2020-06-27 15:06:26,829 WARNING usp.helpers [15023/MainThread]: Request for URL http://www.cmu.edu/sitemap_index.xml failed: 404 Not Found
2020-06-27 15:06:26,830 INFO usp.helpers [15023/MainThread]: Not retrying for URL http://www.cmu.edu/sitemap_index.xml
2020-06-27 15:06:26,831 INFO usp.fetch_parse [15023/MainThread]: Fetching level 0 sitemap from http://www.cmu.edu/.sitemap.xml...
2020-06-27 15:06:26,833 INFO usp.helpers [15023/MainThread]: Fetching URL http://www.cmu.edu/.sitemap.xml...
2020-06-27 15:06:27,156 WARNING usp.helpers [15023/MainThread]: Request for URL http://www.cmu.edu/.sitemap.xml failed: 404 Not Found
2020-06-27 15:06:27,158 INFO usp.helpers [15023/MainThread]: Not retrying for URL http://www.cmu.edu/.sitemap.xml
2020-0

2020-06-27 15:06:31,399 INFO usp.fetch_parse [15023/MainThread]: Fetching level 0 sitemap from http://princeton.edu/sitemap-index.xml...
2020-06-27 15:06:31,400 INFO usp.helpers [15023/MainThread]: Fetching URL http://princeton.edu/sitemap-index.xml...
2020-06-27 15:06:31,684 WARNING usp.helpers [15023/MainThread]: Request for URL http://princeton.edu/sitemap-index.xml failed: 404 Not Found
2020-06-27 15:06:31,687 INFO usp.helpers [15023/MainThread]: Not retrying for URL http://princeton.edu/sitemap-index.xml
2020-06-27 15:06:31,690 INFO usp.fetch_parse [15023/MainThread]: Fetching level 0 sitemap from http://princeton.edu/sitemap.xml...
2020-06-27 15:06:31,690 INFO usp.helpers [15023/MainThread]: Fetching URL http://princeton.edu/sitemap.xml...
2020-06-27 15:06:32,007 WARNING usp.helpers [15023/MainThread]: Request for URL http://princeton.edu/sitemap.xml failed: 404 Not Found
2020-06-27 15:06:32,008 INFO usp.helpers [15023/MainThread]: Not retrying for URL http://princeton.edu/sitema

2020-06-27 15:06:36,374 INFO usp.helpers [15023/MainThread]: Fetching URL http://www.cmu.edu/admin/config/search/xmlsitemap...
2020-06-27 15:06:36,704 WARNING usp.helpers [15023/MainThread]: Request for URL http://www.cmu.edu/admin/config/search/xmlsitemap failed: 404 Not Found
2020-06-27 15:06:36,705 INFO usp.helpers [15023/MainThread]: Not retrying for URL http://www.cmu.edu/admin/config/search/xmlsitemap
2020-06-27 15:06:36,706 INFO usp.fetch_parse [15023/MainThread]: Fetching level 0 sitemap from http://www.cmu.edu/sitemap_index.xml...
2020-06-27 15:06:36,707 INFO usp.helpers [15023/MainThread]: Fetching URL http://www.cmu.edu/sitemap_index.xml...
2020-06-27 15:06:37,043 WARNING usp.helpers [15023/MainThread]: Request for URL http://www.cmu.edu/sitemap_index.xml failed: 404 Not Found
2020-06-27 15:06:37,045 INFO usp.helpers [15023/MainThread]: Not retrying for URL http://www.cmu.edu/sitemap_index.xml
2020-06-27 15:06:37,046 INFO usp.fetch_parse [15023/MainThread]: Fetching level 0 

2020-06-27 15:06:39,645 INFO usp.helpers [15023/MainThread]: Fetching URL http://princeton.edu/sitemap_index.xml.gz...
2020-06-27 15:06:40,014 WARNING usp.helpers [15023/MainThread]: Request for URL http://princeton.edu/sitemap_index.xml.gz failed: 404 Not Found
2020-06-27 15:06:40,015 INFO usp.helpers [15023/MainThread]: Not retrying for URL http://princeton.edu/sitemap_index.xml.gz
2020-06-27 15:06:40,016 INFO usp.fetch_parse [15023/MainThread]: Fetching level 0 sitemap from http://princeton.edu/sitemap-index.xml...
2020-06-27 15:06:40,016 INFO usp.helpers [15023/MainThread]: Fetching URL http://princeton.edu/sitemap-index.xml...
2020-06-27 15:06:40,302 WARNING usp.helpers [15023/MainThread]: Request for URL http://princeton.edu/sitemap-index.xml failed: 404 Not Found
2020-06-27 15:06:40,303 INFO usp.helpers [15023/MainThread]: Not retrying for URL http://princeton.edu/sitemap-index.xml
2020-06-27 15:06:40,304 INFO usp.fetch_parse [15023/MainThread]: Fetching level 0 sitemap from htt

2020-06-27 15:06:45,057 INFO usp.helpers [15023/MainThread]: Not retrying for URL http://www.cmu.edu/sitemap.xml.gz
2020-06-27 15:06:45,057 INFO usp.fetch_parse [15023/MainThread]: Fetching level 0 sitemap from http://www.cmu.edu/admin/config/search/xmlsitemap...
2020-06-27 15:06:45,058 INFO usp.helpers [15023/MainThread]: Fetching URL http://www.cmu.edu/admin/config/search/xmlsitemap...
2020-06-27 15:06:45,398 WARNING usp.helpers [15023/MainThread]: Request for URL http://www.cmu.edu/admin/config/search/xmlsitemap failed: 404 Not Found
2020-06-27 15:06:45,400 INFO usp.helpers [15023/MainThread]: Not retrying for URL http://www.cmu.edu/admin/config/search/xmlsitemap
2020-06-27 15:06:45,400 INFO usp.fetch_parse [15023/MainThread]: Fetching level 0 sitemap from http://www.cmu.edu/sitemap_index.xml...
2020-06-27 15:06:45,401 INFO usp.helpers [15023/MainThread]: Fetching URL http://www.cmu.edu/sitemap_index.xml...
2020-06-27 15:06:45,746 WARNING usp.helpers [15023/MainThread]: Request for 

In [70]:
data

,name,locality,state,country,url,sitemap tree
0,Massachusetts Institute of Technology,"Cambridge, Massachusetts",NaN,United States,https://web.mit.edu/,"IndexWebsiteSitemap(url=https://web.mit.edu/, ..."
1,Harvard University,Cambridge,Massachusetts,United States,http://harvard.edu,"IndexWebsiteSitemap(url=http://harvard.edu/, s..."
2,Stanford University,Stanford,California,United States,http://stanford.edu,"IndexWebsiteSitemap(url=http://stanford.edu/, ..."
3,"University of California, Berkeley",Berkeley,California,United States,http://www.berkeley.edu,IndexWebsiteSitemap(url=http://www.berkeley.ed...
4,Purdue University,West Lafayette,Indiana,United States,http://www.purdue.edu,IndexWebsiteSitemap(url=http://www.purdue.edu/...
5,Cornell University,Ithaca,New York,United States,http://cornell.edu,"IndexWebsiteSitemap(url=http://cornell.edu/, s..."
6,University of Washington,Seattle,Washington,United States,http://www.washington.edu,IndexWebsiteSitemap(url=http://www.washington....
7,Columbia University in the City of New York,New York City,New York,United States,http://columbia.edu,"IndexWebsiteSitemap(url=http://columbia.edu/, ..."
8,University of Michigan,Ann Arbor,Michigan,United States,http://www.umich.edu,"IndexWebsiteSitemap(url=http://www.umich.edu/,..."
9,Penn State University,University Park,Pennsylvania,United States,http://www.psu.edu,"IndexWebsiteSitemap(url=http://www.psu.edu/, s..."


In [71]:
# Search the URL for a given search term
# If we need to implement regex here (and I hope to God we don't) I can implement it
def urlSearch(dfRow, searchTerm):
    sitemap = dfRow['sitemap tree']
    searchResults = [page.url for page in sitemap.all_pages() if searchTerm in page.url]
    return searchResults

# Encapsulate urlSearch() in a lambda so that it has only one parameter exposed
# to do a specific search
facSearch = lambda dfRow: urlSearch(dfRow, 'faculty')

# Search each row of the dataframe and append results to new column
data['search results'] = data.apply(facSearch, axis=1)

# @Emily: This may be a really dumb idea, but could we just choose the result with
# the shortest length???

In [72]:
data[['name', 'search results']]

,name,search results
0,Massachusetts Institute of Technology,[]
1,Harvard University,[https://www.harvard.edu/president/speech/2005...
2,Stanford University,[]
3,"University of California, Berkeley",[]
4,Purdue University,[https://www.purdue.edu/provost/faculty/facult...
5,Cornell University,[]
6,University of Washington,[]
7,Columbia University in the City of New York,[]
8,University of Michigan,[]
9,Penn State University,"[https://www.psu.edu/faculty-and-staff, https:..."


In [ ]:
# Filter search results
def filterSearch(dfRow, terms):
    

In [73]:
# To show you how many URLs we need to sort through
data.loc[13]['search results']

['https://www.nyu.edu/about/careers-at-nyu/faculty-and-researchers.html',
 'https://www.nyu.edu/about/giving/annual-giving/your-impact/faculty.html',
 'https://www.nyu.edu/about/giving/special-campaigns/faculty-admin-staff-campaign.html',
 'https://www.nyu.edu/about/leadership-university-administration/office-of-the-president-emeritus/accomplishments/student-outcomes-and-faculty.html',
 'https://www.nyu.edu/about/leadership-university-administration/office-of-the-president/communications/a-message-to-faculty-from-johnsexton.html',
 'https://www.nyu.edu/about/leadership-university-administration/office-of-the-president/communications/a-message-to-faculty-on-governance-and-faculty-engagement.html',
 'https://www.nyu.edu/about/leadership-university-administration/office-of-the-president/communications/email-about-faculty-advisory-committee-on-the-global-network.html',
 'https://www.nyu.edu/about/leadership-university-administration/office-of-the-president/communications/email-to-fas-facul

In [ ]:
# Write table as-is to CSV so we don't need to rescrape everything
data.to_csv("URL_data_scraped.csv", index=None)